In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

pd.options.display.max_columns = None

resale_1990_csv = './datasets/resale-flat-prices-based-on-approval-date-1990-1999.csv'
resale_2000_csv = './datasets/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv'
resale_2012_csv = './datasets/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv'
resale_2015_csv = './datasets/resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv'

data_1990 = pd.read_csv(resale_1990_csv)
data_2000 = pd.read_csv(resale_2000_csv)
data_2012 = pd.read_csv(resale_2012_csv)
data_2015 = pd.read_csv(resale_2015_csv)


# Data on Longitude and Latitude of HDB Flats
streets_lon_lat = pd.read_csv('./datasets/street_lon_lat.csv', index_col=0)
streets_lon_lat.reset_index(inplace=True)
streets_lon_lat.rename(columns={'index' : 'combined'}, inplace=True)

# Distance from HDB to Stations, taking into consideration station operational year
distance_to_station_month = pd.read_csv('./datasets/flat_station_distance_by_street_month.csv', index_col=0)

# Distance from HDB to Primary/Secondary Schools
flat_primary_distance = pd.read_csv('./datasets/flat_primary_distance.csv', index_col=0)
flat_secondary_distance = pd.read_csv('./datasets/flat_secondary_distance.csv', index_col=0)
flat_primary_distance.reset_index(inplace=True)
flat_secondary_distance.reset_index(inplace=True)
flat_primary_distance.rename(columns={'index' : 'combined'}, inplace=True)
flat_secondary_distance.rename(columns={'index' : 'combined'}, inplace=True)

# HDB Resale Price Index
hdb_rpi = pd.read_csv('./datasets/HDB RPI.csv')

In [2]:
set(data_1990.columns) == set(data_2000.columns) == set(data_2012.columns)

True

In [3]:
[col for col in data_2015.columns if col not in data_2012.columns]

['remaining_lease']

data_1990, data_2000 and data_2012 has the exact same features, but not data_2015. data_2015 has an additional feature - 'remaining_lease'. Since HDB flats have a lease term of 99 years, the same can be calculated for the other datasets by taking [99 - (year sold - year lease commenced)].

## Steps to take:
- Create 'remaining_lease' feature for data_1990, data_2000 and data_2012

In [2]:
lease_col_loc = list(data_2015.columns).index('remaining_lease')

data_1990.insert(loc=lease_col_loc, column='remaining_lease', value=99 - (pd.to_datetime(data_1990['month']).apply(lambda x: x.year) - data_1990['lease_commence_date']))
data_2000.insert(loc=lease_col_loc, column='remaining_lease', value=99 - (pd.to_datetime(data_2000['month']).apply(lambda x: x.year) - data_2000['lease_commence_date']))
data_2012.insert(loc=lease_col_loc, column='remaining_lease', value=99 - (pd.to_datetime(data_2012['month']).apply(lambda x: x.year) - data_2012['lease_commence_date']))

all_data = data_1990.append(data_2000, ignore_index=True).append(data_2012, ignore_index=True).append(data_2015, ignore_index=True)
all_data.replace({'MULTI GENERATION' : 'MULTI-GENERATION'}, inplace=True)

all_data['month_sold'] = pd.to_datetime(all_data['month']).apply(lambda x: x.month).astype(str)
all_data['year_sold'] = pd.to_datetime(all_data['month']).apply(lambda x: x.year).astype(str)

print('Shape of merged Dataset:\t{}'.format(all_data.shape))

Shape of merged Dataset:	(785798, 13)


In [5]:
all_data['combined'] = all_data['street_name'] + ' BLK ' + all_data['block']
all_data = all_data.merge(streets_lon_lat, how='left', on='combined').merge(distance_to_station_month, how='left', on=['month', 'combined'])
all_data = all_data.merge(flat_primary_distance[['combined', 'Primary School', 'Primary Distance']], how='left', on='combined').merge(flat_secondary_distance[['combined', 'Secondary School', 'Secondary Distance']], how='left', on='combined')

hdb_rpi['Deflator'] = hdb_rpi['index'].apply(lambda x: hdb_rpi.loc[hdb_rpi.index[-1], 'index']/x)

all_data['quarter'] = all_data['year_sold'].astype(str) + '-' + all_data['month_sold'].apply(lambda x: 'Q1' if int(x) <= 3 else ('Q2' if int(x) <= 6 else ('Q3' if int(x) <= 9 else 'Q4'))).astype(str)
all_data = all_data.merge(hdb_rpi[['quarter', 'Deflator']], how='left', on='quarter')
all_data['Deflated Resale Price'] = all_data['resale_price'] * all_data['Deflator']
all_data.drop(columns=['combined', 'quarter', 'Deflator'], inplace=True)

all_data['Primary Distance Categorised'] = all_data['Primary Distance'].apply(lambda x: '<= 100m' if x <= 100 else ('101m - 200m' if x <= 200 else ('201m - 300m' if x <= 300 else ('301 - 400m' if x <= 400 else ('401m - 500m' if x <= 500 else '> 500m')))))
all_data['Secondary Distance Categorised'] = all_data['Secondary Distance'].apply(lambda x: '<= 100m' if x <= 100 else ('101m - 200m' if x <= 200 else ('201m - 300m' if x <= 300 else ('301 - 400m' if x <= 400 else ('401m - 500m' if x <= 500 else '> 500m')))))

all_data.columns = [col.replace('_',' ').title() for col in all_data.columns]

all_data['Flat Model'] = all_data['Flat Model'].apply(lambda x: x.upper().replace('.','').replace('-',' '))

Storey Range for HDB sold between March and May 2012 seems to follow a different scale (i.e. in step size of 5 levels - 01 TO 05, instead of the usual 3 level step size - 01 TO 03). It is difficult to merge them together on the same scale unless the stories are categorised based on common multiples, such as 01 TO 15, 16 TO 30, etc. However this would likely cause the feature to lose precious information and thus we would just drop sales between March and May 2012 (0.87% of all data).

Furthermore, as the HDB RPI ends on 2018 Q3, deflated resale prices are unavailable for sales after that (2018 Q4 data makes up 0.2% of all data). Therefore, these will be dropped as well.

In [6]:
all_data[all_data['Month'].isin(['2012-03', '2012-04', '2012-05'])]['Storey Range'].value_counts().sort_index()

01 TO 05    2700
06 TO 10    2474
11 TO 15    1259
16 TO 20     265
21 TO 25      92
26 TO 30      39
31 TO 35       2
36 TO 40       7
Name: Storey Range, dtype: int64

In [7]:
all_data[~all_data['Month'].isin(['2012-03', '2012-04', '2012-05'])]['Storey Range'].value_counts().sort_index()

01 TO 03    161500
04 TO 06    200071
07 TO 09    180656
10 TO 12    152762
13 TO 15     48385
16 TO 18     17602
19 TO 21      8668
22 TO 24      5497
25 TO 27      2235
28 TO 30       869
31 TO 33       193
34 TO 36       202
37 TO 39       191
40 TO 42        95
43 TO 45        12
46 TO 48        15
49 TO 51         7
Name: Storey Range, dtype: int64

In [8]:
print('Observations between March and May 2012 makes up {}% of the entire dataset.'.format(round(all_data[all_data['Month'].isin(['2012-03', '2012-04', '2012-05'])].count()['Month'] / all_data.shape[0] * 100, 2)))

Observations between March and May 2012 makes up 0.87% of the entire dataset.


In [9]:
all_data.isna().sum()

Month                                0
Town                                 0
Flat Type                            0
Block                                0
Street Name                          0
Storey Range                         0
Floor Area Sqm                       0
Flat Model                           0
Lease Commence Date                  0
Remaining Lease                      0
Resale Price                         0
Month Sold                           0
Year Sold                            0
Latitude                             0
Longitude                            0
Distance To Nearest Station          0
No. Of Stations Within 500M          0
Primary School                       0
Primary Distance                     0
Secondary School                     0
Secondary Distance                   0
Deflated Resale Price             2000
Primary Distance Categorised         0
Secondary Distance Categorised       0
dtype: int64

In [10]:
print('Observations after 2018 Q3 makes up {}% of all data.'.format(round(all_data[all_data['Month'] == '2018-10'].count()['Month'] / all_data.shape[0] * 100, 2)))

Observations after 2018 Q3 makes up 0.25% of all data.


## Dropping these observations

In [11]:
all_data.drop(index=all_data[all_data['Month'].isin(['2012-03', '2012-04', '2012-05'])].index, inplace=True)
all_data.dropna(inplace=True)
all_data.reset_index(inplace=True, drop=True)
all_data = all_data.reindex(columns=['Month', 'Month Sold', 'Year Sold', 'Town', 'Flat Type', 'Flat Model', 'Block', 'Street Name', 'Longitude', 'Latitude', 'Storey Range', 'Floor Area Sqm', 'Lease Commence Date', 'Remaining Lease', 'Distance To Nearest Station', 'No. Of Stations Within 500M', 'Primary School', 'Primary Distance', 'Primary Distance Categorised', 'Secondary School', 'Secondary Distance', 'Secondary Distance Categorised', 'Resale Price', 'Deflated Resale Price'])

In [14]:
midpoint = int(all_data.shape[0] / 2)

all_data.loc[all_data.index[:midpoint], :].to_csv('./datasets/merged_dataset_part1.csv')
all_data.loc[all_data.index[midpoint:], :].to_csv('./datasets/merged_dataset_part2.csv')

## Final Dataset

In [15]:
all_data.head(2)

,Month,Month Sold,Year Sold,Town,Flat Type,Flat Model,Block,Street Name,Longitude,Latitude,Storey Range,Floor Area Sqm,Lease Commence Date,Remaining Lease,Distance To Nearest Station,No. Of Stations Within 500M,Primary School,Primary Distance,Primary Distance Categorised,Secondary School,Secondary Distance,Secondary Distance Categorised,Resale Price,Deflated Resale Price
0,1990-01,1,1990,ANG MO KIO,1 ROOM,IMPROVED,309,ANG MO KIO AVE 1,103.84411,1.364329,10 TO 12,31.0,1977,86,> 500m,0,TECK GHEE PRIMARY SCHOOL,722.787614,> 500m,ANG MO KIO SECONDARY SCHOOL,375.846945,301 - 400m,9000.0,48740.740741
1,1990-01,1,1990,ANG MO KIO,1 ROOM,IMPROVED,309,ANG MO KIO AVE 1,103.84411,1.364329,04 TO 06,31.0,1977,86,> 500m,0,TECK GHEE PRIMARY SCHOOL,722.787614,> 500m,ANG MO KIO SECONDARY SCHOOL,375.846945,301 - 400m,6000.0,32493.827160
